In [1]:
import keras
import numpy as np
from keras.datasets import mnist
from sklearn.metrics import accuracy_score

In [2]:
from micrograd.engine import Value
from micrograd.nn import MLP

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.asarray(x_train, dtype = np.float32) / 255.0
x_train = x_train.reshape(60000, 784)
x_test = np.asarray(x_test, dtype = np.float32) / 255.0
x_test = x_test.reshape(10000, 784)
y_train = keras.utils.to_categorical(y_train)

In [4]:
epochs = 5
batch_size = 32
learning_rate = 0.01
model = MLP(784, [128, 10])

pre = 0
cur = batch_size
index = np.random.permutation(x_train.shape[0])

for epoch in range(epochs):
    print(f'epoch {epoch}, accuracy: {(accuracy_score(np.argmax(model(x_test).data, axis = 1), y_test) * 100):.2f}%')

    for step in range(int(x_train.shape[0] / batch_size)):
        ri = index[pre:cur]
        Xb, yb = Value(x_train[ri]), Value(y_train[ri])

        y_predW = model(Xb)
        probs = y_predW.softmax()
        log_probs = probs.log()
        zb = yb * log_probs
        outb = zb.reduce_sum(axis = 1)
        finb = -outb.reduce_sum()

        model.zero_grad()
        finb.backward()

        for p in model.parameters():
            if len(p.data.shape) > 1:
                p.data -= learning_rate * p.grad
            else:
                p.data -= learning_rate * p.grad.sum(axis = 0)

        if cur == x_train.shape[0]:
            pre = 0
            cur = batch_size
        else:
            pre = cur

            if cur <= x_train.shape[0] - batch_size:
                cur += batch_size
            else:
                cur = x_train.shape[0]

print(f'epoch {epochs}, accuracy: {(accuracy_score(np.argmax(model(x_test).data, axis = 1), y_test) * 100):.2f}%')

epoch 0, accuracy: 13.29%
epoch 1, accuracy: 96.35%
epoch 2, accuracy: 96.89%
epoch 3, accuracy: 97.33%
epoch 4, accuracy: 97.49%
epoch 5, accuracy: 97.59%
